In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import gc
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
from data import MemmapDataset
from models import *
from torchvision.models import ResNet50_Weights
from torchgeo.models import resnet18, resnet50, get_weight
from typing import List

In [3]:
# Config
DATA_SPLIT = 0.90
INIT_LR = 0.005
NUM_EPOCHS = 20
BATCH_SIZE = 32
INPUT_IMAGE_SIZE = 224
THRESHOLD = 0.5
LOSS = JaccardLoss()

# define the path to each directory
BASE_DIR = "C:\\Users\\gwrye\\OneDrive\\Desktop\\Drone_Dataset"
IMAGE_DIR = BASE_DIR + "\\dataset_images.npy"
LABEL_DIR = BASE_DIR + "\\dataset_labels.npy"

dataset = MemmapDataset(np.load(IMAGE_DIR, 'r'), np.load(LABEL_DIR, 'r'))
print(f"Dataset containing {len(dataset)} images loaded.")

# Setup the device to be used for training and evaluation
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using CUDA device.")
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using Apple Metal Performance Shaders (MPS) device.\n")
else:
    DEVICE = torch.device("cpu")
    print("WARNING: No GPU found. Defaulting to CPU.")

trainDS, testDS = dataset.split(0.9)                           
# calculate steps per epoch for training and test set #config
TRAIN_STEPS = len(trainDS) // BATCH_SIZE
TEST_STEPS = len(testDS) // BATCH_SIZE

# create the training and test data loaders #config
trainLoader = DataLoader(trainDS, shuffle=True,
                         batch_size=BATCH_SIZE)
testLoader = DataLoader(testDS, shuffle=False,
                        batch_size=BATCH_SIZE)


del trainDS, testDS, dataset

# Training Functions
def train(model, trainLoader : DataLoader, testLoader : DataLoader, lossFunc, NUM_EPOCHS=NUM_EPOCHS, print_all_epochs = False):
  opt = Adam(model.parameters(), lr=INIT_LR)
  # loop over epochs #config
  print("[INFO] training the network...")
  training_loss = []
  all_metrics = []

  for e in tqdm(range(NUM_EPOCHS)):
    # set the model in training mode
    model.train()
    totalTrainLoss = 0

    # loop over the training set
    for (i, (x, y)) in enumerate(trainLoader):
      # send the input to the device
      x = x.to(DEVICE)
      y = y.to(DEVICE).float()
      # perform a forward pass and calculate the training loss
      pred = model(x)
      if isinstance(pred, tuple):
        pred = pred[0]
      loss = lossFunc(pred, y)

      # first, zero out any previously accumulated gradients, then
      # perform backpropagation, and then update model parameters
      opt.zero_grad()
      loss.backward()
      opt.step()

      # add the loss to the total training loss so far
      totalTrainLoss += loss.item()
    # calculate the average training
    avgTrainLoss = totalTrainLoss / TRAIN_STEPS
    training_loss.append(avgTrainLoss)

    # Evaluate on test dataset
    metrics = evaluate(model, testLoader, lossFunc)
    all_metrics.append(metrics)
    avgTestLoss = metrics['Loss']

    if (e + 1) % 5 == 0 or e == 0 or print_all_epochs:
      # print the model training and validation information
      print("EPOCH: {}/{}".format(e + 1, NUM_EPOCHS)) #config
      print("Train loss: {:.6f}, Test loss: {:.4f}".format(
          avgTrainLoss, avgTestLoss))
      print("\nValidation Metrics:")
      for k, v in metrics.items():
          if k != 'Loss':
            print(f"{k}: {v}")
      print("\n")
  return training_loss, all_metrics

def evaluate(model: nn.Module, dataloader: DataLoader, loss_func):
    model.eval()
    total_loss = 0
    total_TP = 0
    total_FP = 0
    total_FN = 0
    total_TN = 0
    total_landmass_captured = 0
    total_landmass_actual = 0

    with torch.no_grad():
        for (x, y) in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE).float()
            
            pred = model(x)
            if isinstance(pred, tuple):
                pred = pred[0]
            loss = loss_func(pred, y)
            total_loss += loss.item()

            pred = torch.sigmoid(pred).view(-1)
            y = y.view(-1)
            
            TP = (pred * y).sum().item()
            FP = ((1 - y) * pred).sum().item()
            FN = (y * (1 - pred)).sum().item()
            TN = ((1 - y) * (1 - pred)).sum().item()
        
            total_TP += TP
            total_FP += FP
            total_FN += FN
            total_TN += TN

            total_landmass_actual += y.sum().item()
            total_landmass_captured += pred.sum().item()

    total_landmass_captured = total_landmass_captured / total_landmass_actual if total_landmass_actual > 0 else 0
    avg_loss = total_loss / len(dataloader)
    precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
    recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    iou = total_TP / (total_TP + total_FP + total_FN) if (total_TP + total_FP + total_FN) > 0 else 0
    accuracy = (total_TP + total_TN) / (total_TP + total_FP + total_FN + total_TN) if (total_TP + total_FP + total_FN + total_TN) > 0 else 0
    specificity = total_TN / (total_TN + total_FP) if (total_TN + total_FP) > 0 else 0

    metrics = {
        'Landmass Captured': total_landmass_captured,
        'Loss': avg_loss,
        'Precision': precision,
        'Recall': recall,
        'f1_score': f1_score,
        'IOU': iou,
        'Accuracy': accuracy,
        'Specificity': specificity
    }

    return metrics



# Plotting functions
def plot_losses(title, training_loss, validation_loss, training_time=None, y_max=0.3):
  # scale losses to fit graph
  valid_loss = [min(x,y_max) for x in validation_loss]
  train_loss = [min(x, y_max) for x in training_loss]


  plt.figure()
  plt.plot(np.arange(1, NUM_EPOCHS+1), train_loss, label="train_loss")
  plt.plot(np.arange(1, NUM_EPOCHS+1), valid_loss, label="valid_loss")
  plt.title(title)
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend(loc="upper right")
  if training_time is not None:
    plt.text(0, 0.3, f"Training Time: {training_time}")

  step = y_max / 10
  yticks = np.arange(0, y_max+step, step)  # Generate ticks from 0.025 to 0.3 with step 0.025
  plt.yticks(yticks)

  xticks = np.arange(2, NUM_EPOCHS+2, 2)  # Generate ticks from 0 to num_epochs with step 2
  plt.xticks(xticks)
  
  plt.show()

def plot_loss_comparison(title, training_losses1, training_losses2, validation_losses1, validation_losses2, compare1 = "Satellite", compare2 = "ImageNet", y_max=0.3):
    # scale losses to fit graph
    valid_loss_sat = [min(x, y_max) for x in validation_losses1]
    train_loss_sat = [min(x, y_max) for x in training_losses1]
    valid_loss_img = [min(x, y_max) for x in validation_losses2]
    train_loss_img = [min(x, y_max) for x in training_losses2]
    
    plt.figure()
    plt.plot(np.arange(0, NUM_EPOCHS), train_loss_sat, label=f"Training loss {compare1}", color='orange')
    plt.plot(np.arange(0, NUM_EPOCHS), valid_loss_sat, label=f"Validation loss {compare1}", color='orange', linestyle='dashed')
    plt.plot(np.arange(0, NUM_EPOCHS), train_loss_img, label=f"Training loss {compare2}", color='teal')
    plt.plot(np.arange(0, NUM_EPOCHS), valid_loss_img, label=f"Validation loss {compare2}", color='teal', linestyle='dashed')
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc="upper right")
    
    yticks = np.arange(0.025, 0.325, 0.025)  # Generate ticks from 0.025 to 0.3 with step 0.025
    plt.yticks(yticks)
    
    xticks = np.arange(2, NUM_EPOCHS+2, 2)  # Generate ticks from 0 to num_epochs with step 2
    plt.xticks(xticks)
    
    plt.show()

def plot_metrics(title: str, metric_dict: dict, metrics: List = ['Precision', 'Recall', 'IOU']):
    plt.figure()
    for metric in metrics:
        plt.plot(np.arange(0, NUM_EPOCHS), [x[metric] for x in metric_dict], label=metric)
    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.legend(loc="lower right")

    yticks = np.arange(0.0, 1.1, 0.1)
    plt.yticks(yticks)

    xticks = np.arange(2, NUM_EPOCHS+2, 2)
    plt.xticks(xticks)
    
    plt.show()


def plot_comparison_metrics(title, metrics: List[List[dict]], titles: List[str],
                             metrics_wanted = ['Precision', 'Recall', 'IOU'], x_label='Metrics', y_label = 'Values', y_lim = 1.1, 
                             size = (10.0, 6.0), single_metric=False):
    plt.figure(figsize=size)
    
    if single_metric:
        for i in range(len(titles)):
            plt.bar(titles[i], metrics[i][-1][metrics_wanted[0]])
    else:
        extracted_metrics = []
        for i in range(len(titles)):
            metrics_add = []
            for k in metrics[i][-1]:
                if k in metrics_wanted:
                    metrics_add.append(metrics[i][-1][k])
            extracted_metrics.append(metrics_add)

        print(extracted_metrics)

        # Create bar positions
        bar_width = 0.8 / len(titles)  # Adjust bar width based on number of titles
        r = np.arange(len(metrics_wanted))
        
        for i in range(len(titles)):
            plt.bar([x + i * bar_width for x in r], extracted_metrics[i], width=bar_width, edgecolor='grey', label=titles[i])
        plt.xticks([r + bar_width * (len(titles) / 2) for r in range(len(metrics_wanted))], metrics_wanted)

        plt.legend()
    
    # Adding labels
    plt.xlabel(x_label, fontweight='bold')
    plt.ylabel(y_label, rotation=0, labelpad=len(y_label)*2)
    plt.title(title)
    
    plt.ylim(0, y_lim)
    plt.show()

Dataset containing 680079 images loaded.
Using CUDA device.


In [5]:
BATCH_SIZE = 32
NUM_EPOCHS = 10

model = ResNet_UNet_NoSkip(input_image_size=INPUT_IMAGE_SIZE).to(DEVICE)
training_loss, metrics = train(model, trainLoader, testLoader, LOSS, NUM_EPOCHS=NUM_EPOCHS, print_all_epochs=True)
valid_loss = [x['Loss'] for x in metrics]

[INFO] training the network...


 10%|█         | 1/10 [14:11<2:07:44, 851.64s/it]

EPOCH: 1/10
Train loss: 0.262424, Test loss: 0.2248

Validation Metrics:
Landmass Captured: 1.1012208139551096
Precision: 0.8348171414415761
Recall: 0.9193180245485095
f1_score: 0.8750322782522006
IOU: 0.7778287868497347
Accuracy: 0.9056962302127051
Specificity: 0.8980626501886066




 20%|██        | 2/10 [28:52<1:55:50, 868.84s/it]

EPOCH: 2/10
Train loss: 0.207344, Test loss: 0.2061

Validation Metrics:
Landmass Captured: 1.043541471123337
Precision: 0.8681107048557829
Recall: 0.9059095242356114
f1_score: 0.8866074272699042
IOU: 0.7963116056144485
Accuracy: 0.9167800261395966
Specificity: 0.9228717968563385




...

EPOCH: 8/10
Train loss: 0.146378, Test loss: 0.1438

Validation Metrics:
Landmass Captured: 1.0352720756516731
Precision: 0.9080867588823465
Recall: 0.9401168643013899
f1_score: 0.9238242643453727
IOU: 0.8584325345185555
Accuracy: 0.9443200750496308
Specificity: 0.946675531714635




 90%|█████████ | 9/10 [2:11:30<14:37, 877.94s/it]

EPOCH: 9/10
Train loss: 0.140711, Test loss: 0.1458

Validation Metrics:
Landmass Captured: 1.029791973452709
Precision: 0.9093864521484633
Recall: 0.9364788682546468
f1_score: 0.9227338377299592
IOU: 0.8565513983893756
Accuracy: 0.9436751059818781
Specificity: 0.9477078388568785




100%|██████████| 10/10 [2:26:04<00:00, 876.49s/it]

EPOCH: 10/10
Train loss: 0.137040, Test loss: 0.1360

Validation Metrics:
Landmass Captured: 1.02064103893727
Precision: 0.9189703475612869
Recall: 0.9379388507000025
f1_score: 0.9283577166035776
IOU: 0.8662944071796755
Accuracy: 0.9480102057788823
Specificity: 0.9536541389178916




In [6]:
torch.save(model.state_dict(), "resnet_unet_noskip_224.pth")

In [7]:
# Number of parameters
res18 = ResNet_UNet()
NoSkip = ResNet_UNet_NoSkip(input_image_size=INPUT_IMAGE_SIZE)

res18_params = sum(p.numel() for p in res18.parameters())
NoSkip_params = sum(p.numel() for p in NoSkip.parameters())
print(f"ResNet18 UNet has {res18_params} parameters.")
print(f"ResNet18 UNet NoSkip has {NoSkip_params} parameters.")

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor